# STEP 9 함수를 더 편리하게
- 현재 DeZero가 역전파를 해낼 수 있게 완료함
- 또한 Define-by-Run이라고 하는 전체 계산의 각 조각들을 런타임에 '연결'해내는 능력도 갖춤
- 이번 step에서는 사용에 불편한 부분을 개선할 예정임

## 9.1 파이썬 함수로 이용하기
- 지금까지 DeZero는 함수를 '파이썬 클래스'로 정의해 사용했음
- 예를 들어 Square 클래스를 사용하는 계산을 하기 위해서는 코드를 아래와 같이 작성해야 했음

In [ ]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

- Square 클래스의 인스턴스 생성 후, 그 인스턴스를 호출하는 **두 단계**로 구분되었음
- y = Square()(X) 형태로 적을 수 있지만 모양새가 좋지 않음
- 따라서 '파이썬 함수'를 지원하는 방식을 사용할 것임

In [ ]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

- 이를 통해 DeZero 함수를 파이썬 함수로 이용 가능해짐
- 아래와 같이 한 줄로 적을 수도 있음

In [ ]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [ ]:
# 사용 예시
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)
# >>> 3.297442541400256

## 9.2 backward 메서드 간소화
- 위의 사용 예시 코드에서 `y.grad = np.array(1.0)` 부분을 생략하려 함

In [ ]:
class Variable:
    # ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x,y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

- 위의 코드와 같이, 만약 변수의 grad가 None이면 자동으로 미분 값을 생성함
- `np.ones_like(self.data)`는 self.data와 형상과 데이터 타입이 같은 ndarray 인스턴스를 생성
- 이제 계산 하고 난 뒤 최종 출력 변수에서 backward 메서드를 호출하는 것만으로 미분값이 구해짐

In [ ]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)
# >>> 3.297442541400256

## 9.3 ndarray만 취급하기
- DeZero의 Variable은 데이터로 ndarray 인스턴스만 취급하게끔 의도했음
- 하지만 실수로 inf, float 같은 데이터 타입 입력 시 오류를 일으킬 것임

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None

- 그런데 이렇게 바꿀 경우, 주의할 것이 하나 생김
- 1차원 ndarray(ex. `np.array([1.0])`)를 제곱하면 결과의 데이터 타입도 ndarray가 됨
- 그러나 0차원 ndarray(ex. `np.array(1.0)`)를 제곱하면 np.float64 데이터 타입이 됨
- 이는 넘파이가 의도한 동작이며, DeZero에선 입력과 출력 모두 ndarrary라 가정했으므로 대처 해야함

In [ ]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [ ]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        # 추가한 부분
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

# STEP 10 테스트
## 10.1 파이썬 단위 테스트
- 표준 라이브러리에 포함된 unittest 사용

In [ ]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

- 지켜야할 규칙은, 테스트 할 때 이름이 test로 시작하는 메서드를 만들고 그 안에 테스트할 내용을 적어야한다는 것임
- CLI에서 `python -m unittest '파일명'.py`로 실행 가능

## 10.2 square 함수의 역전파 테스트

In [ ]:
class SquareTest(unittest.TestCase):
    # ...
    
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

## 10.3 기울기 확인을 이용한 자동 테스트
- 기울기 확인이란 수치 미분으로 구한 결과와 역전파로 구한 결과를 비교하여 그 차이가 크면 역전파에 문제가 있다고 판단하는 검증 기법

In [ ]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

class SquareTest(unittest.TestCase):
    # ...
    
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        # np.allclose(a,b) : 가까운지 가깝지 않은 지 판정해 bool 값 return
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)